In [1]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import flowkit as fk
from tensorflow import keras
windowsize = 9
from tensorflow.keras import layers

from sklearn.utils import class_weight

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [2]:
adpos = pd.read_csv("DeepLearning_for_ImagingFlowCytometry\data/adpos.csv")

In [3]:
adpos

,SS-A,FITC-A,PC5-A,APC-750-A,SS-H,FITC-H,PC5-H,APC-750-H
0,1.250060,0.042546,0.044085,0.0,0.330808,0.008921,0.026113,0.0
1,4.736633,0.148447,0.020853,0.0,0.257450,0.010920,0.015950,0.0
2,2.746320,0.074259,0.017845,0.0,0.548905,0.009920,0.014942,0.0
3,1.196145,0.037580,0.022857,0.0,0.207417,0.007924,0.016960,0.0
4,0.686035,0.046508,0.022857,0.0,0.173742,0.007924,0.017972,0.0
...,...,...,...,...,...,...,...,...
13552,1.797759,0.050430,0.028879,0.0,0.307258,0.007924,0.015950,0.0
13553,2.249379,0.076265,0.016865,0.0,0.331993,0.011922,0.015950,0.0
13554,4.265646,0.080175,0.017850,0.0,0.879576,0.011922,0.016960,0.0
13555,4.071285,0.067271,0.020847,0.0,0.813439,0.009920,0.018985,0.0


In [5]:
adpos2 =  pd.read_csv("DeepLearning_for_ImagingFlowCytometry\data/adpos2.csv")

In [6]:
adpos3 =  pd.read_csv("DeepLearning_for_ImagingFlowCytometry\data/adpos3.csv")
adpos4 =  pd.read_csv("DeepLearning_for_ImagingFlowCytometry\data/adpos4.csv")

In [7]:
alladpos = pd.concat([adpos, adpos2, adpos3, adpos4], ignore_index = True)

In [8]:
sum = 0
allpossc = []
for x in alladpos['SS-A']:
    sum += int(x)
    allpossc.append(x*20)
    
print(sum / len(alladpos['SS-A']))
print(len(allpossc))

2.9455756207674946
44300


In [9]:
freshnegpath = "DeepLearning_for_ImagingFlowCytometry\data/FreshNegative.csv"
freshneg = pd.read_csv(freshnegpath)
frozennegpath = "DeepLearning_for_ImagingFlowCytometry\data/FrozenNegative.csv"
frozenneg = pd.read_csv(frozennegpath)

In [10]:
freshneg

,Time,FSC-A,FSC-W,FSC-H,SSC-A,SSC-W,SSC-H,FITC-A,PE-A,PerCP-Cy5_5-A,PE-Cy7-A,APC-A,APC-H7-A,V450-A,V500-A
0,0.001000,164.886969,74.770336,144.523,23.804318,70.846492,22.020,0.337779,0.221039,0.138748,0.083249,-0.005416,0.058670,0.459613,0.465334
1,0.007000,15.917076,74.568695,13.989,30.295795,72.626570,27.338,0.046887,0.146403,0.212427,0.054542,0.104703,0.059572,0.298462,0.335651
2,0.007000,143.825203,72.007641,130.899,80.493750,73.051094,72.213,0.405717,1.175049,2.281203,0.418157,0.595722,0.437766,2.915015,3.335533
3,0.011000,88.718930,83.574586,69.570,13.293941,72.900320,11.951,0.091861,0.052628,0.091861,-0.027750,-0.006318,0.007221,0.343280,0.337558
4,0.011000,157.695219,74.516109,138.691,40.072246,72.054617,36.447,0.167454,0.176066,0.101429,0.006698,0.032494,0.019857,0.475824,0.464381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,37.207000,108.282547,79.534711,89.224,22.706775,74.858453,19.879,0.247832,0.094731,0.136834,0.030620,0.030689,0.000000,0.386189,0.381422
19996,37.209001,66.599695,76.627070,56.960,52.251422,74.769078,45.799,0.076550,0.120567,0.264099,0.037318,0.085748,-0.003610,0.433867,0.449124
19997,37.212000,157.070578,76.867422,133.916,42.518988,70.260320,39.660,0.342563,0.231565,0.062197,0.025836,0.013539,0.057767,0.617903,0.450078
19998,37.213000,46.803934,103.201086,29.722,12.280605,78.888625,10.202,0.119610,0.131093,0.083249,0.028706,0.083040,-0.011734,0.413843,0.253645


In [11]:
alladneg = pd.concat([freshneg, frozenneg], ignore_index = True)

In [12]:
alladneg

,Time,FSC-A,FSC-W,FSC-H,SSC-A,SSC-W,SSC-H,FITC-A,PE-A,PerCP-Cy5_5-A,PE-Cy7-A,APC-A,APC-H7-A,V450-A,V500-A,Alexa 488-A,APC-Cy7-A
0,0.001000,164.886969,74.770336,144.523,23.804318,70.846492,22.020,0.337779,0.221039,0.138748,0.083249,-0.005416,0.058670,0.459613,0.465334,NaN,NaN
1,0.007000,15.917076,74.568695,13.989,30.295795,72.626570,27.338,0.046887,0.146403,0.212427,0.054542,0.104703,0.059572,0.298462,0.335651,NaN,NaN
2,0.007000,143.825203,72.007641,130.899,80.493750,73.051094,72.213,0.405717,1.175049,2.281203,0.418157,0.595722,0.437766,2.915015,3.335533,NaN,NaN
3,0.011000,88.718930,83.574586,69.570,13.293941,72.900320,11.951,0.091861,0.052628,0.091861,-0.027750,-0.006318,0.007221,0.343280,0.337558,NaN,NaN
4,0.011000,157.695219,74.516109,138.691,40.072246,72.054617,36.447,0.167454,0.176066,0.101429,0.006698,0.032494,0.019857,0.475824,0.464381,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,39.144001,87.748656,103.863172,55.368,30.061379,113.335016,17.383,NaN,0.330714,0.100896,0.011211,0.029810,NaN,0.599553,0.590631,1.860969,-0.011559
44996,39.144001,65.284594,91.669516,46.673,26.183426,82.768523,20.732,NaN,0.222345,0.139199,0.027092,0.022510,NaN,0.413977,0.435390,1.057539,0.012167
44997,39.153000,94.935320,85.714617,72.586,33.266695,87.356898,24.957,NaN,0.126120,0.158818,0.010276,0.045628,NaN,0.651300,0.550482,0.508216,0.021293
44998,39.154001,65.739984,94.694930,45.497,23.966518,84.567367,18.573,NaN,0.140133,0.160686,0.021487,0.026160,NaN,0.296208,0.327434,0.373689,0.026160


In [14]:
sum = 0
allnegsc = []
for x in alladneg['SSC-A']:
    sum += int(x)
    allnegsc.append(x*20) # scatter conversion
    
print(sum / len(alladneg['SSC-A']))
print(len(allnegsc))

70.22442222222222
45000


In [15]:
import math

consec = pd.DataFrame()
rows, cols = (56, 784)
negarr = [[0.0]*cols]*rows
negarr = np.array(negarr)

posarr = [[0.0]*cols]*rows
posarr = np.array(posarr)


#19600
for x in range(43904):
    negarr[x//784][x % 784] = alladneg['SSC-A'][x]
    
for x in range(43904):
    posarr[x//784][x % 784] = allpossc[x]
    
        


print(posarr)

[[ 25.00120605  94.73265625  54.9263916  ... 521.85613281 134.93474609
   81.20952148]
 [ 36.75010986  38.68217041  27.40068359 ...  59.08756836  57.12021973
  366.51      ]
 [ 16.00778931  44.71537109  20.09383545 ...  80.7370459   26.92708984
   15.22270752]
 ...
 [ 11.58981201 518.46070313  17.2773584  ...  36.78266113  25.85379883
  166.45279297]
 [ 11.50072266  21.75421387  23.12070801 ...  11.90499023  15.62934814
   20.49458496]
 [ 22.02013184  38.27886475 128.28395508 ... 190.73183594  13.92727661
   91.24111328]]


In [16]:
print(negarr)

[[2.38043184e+01 3.02957949e+01 8.04937500e+01 ... 6.01810898e+01
  1.26646969e+02 2.62143000e+02]
 [2.27948086e+01 1.84524844e+01 2.58711797e+01 ... 1.87500742e+01
  5.16562422e+01 2.63123027e+01]
 [2.80174629e+01 1.36533453e+02 2.62143000e+02 ... 2.58022852e+01
  7.21162578e+01 3.30669219e+01]
 ...
 [8.83072891e+01 1.95102812e+01 2.15106375e+02 ... 2.40865656e+02
  4.69371563e+01 5.71351172e+01]
 [5.22183125e+01 4.23977734e+01 2.41888633e+01 ... 2.65487070e+01
  6.83962266e+01 2.42897587e-02]
 [2.62143000e+02 2.44417563e+02 3.21204043e+01 ... 3.48688828e+01
  2.51062695e+01 9.32829531e+01]]


In [17]:
import numpy as np
from PIL import Image as im
  
# define a main function
def imageconv(x):
  
    # create a numpy array from scratch
    # using arange function.
    # 1024x720 = 737280 is the amount 
    # of pixels.
    # np.uint8 is a data type containing
    # numbers ranging from 0 to 255 
    # and no non-negative integers
    array = negarr[x]
      
    # check type of array
    print(type(array))
      
    # our array will be of width 
    # 737280 pixels That means it 
    # will be a long dark line
    print(array.shape)
      
    # Reshape the array into a 
    # familiar resoluition
    array = np.reshape(array, (28,28))
      
    # show the shape of the array
    print(array.shape)
  
    # show the array
    print(array)
      
    # creating image object of
    # above array
    data = im.fromarray(array)
    imrgb = data.convert('RGB')
    # saving the final output 
    # as a PNG file
    #imrgb.save('adneg' + str(x) +'.png')
    
    
    
for x in range(56):
    imageconv(x)
  
# driver code
if __name__ == "__main__":
    
  # function call
  main()

<class 'numpy.ndarray'>
(784,)
(28, 28)
[[ 23.80431836  30.29579492  80.49375     13.29394141  40.07224609
   47.17516797  38.89815234  48.38562109  29.21547852  22.06758008
   68.55188281  21.9202207   28.83176758  23.02254687  21.3432207
   30.45750781  14.43071582  48.84492578  35.05819141 131.13570312
  262.143      262.143       34.54147656  22.47425391  26.43574023
   75.80121094  40.01291797  70.62926563]
 [ 46.64314453  53.52311328  40.81574219  41.69607031  22.76036133
  262.143       21.05615625  17.8190293   51.17588672  31.30530469
   99.38161719  21.47814063  44.42605078   7.81101611  24.86836914
    5.25805859  36.59685547  10.49315234  24.13635547  31.16081641
   31.50146484  30.07762695  43.18019141  40.31337891 262.143
   40.4674375   36.23323828  24.81669727]
 [ 39.87608203 141.36953125  48.18372266  33.40757031  27.39549023
    5.73075781  81.86782813  25.59272852  63.19813281  38.29244531
   40.99658984  19.48878711  49.0028125   52.92984766 262.143
   53.84080078  

NameError: name 'main' is not defined

In [18]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.layers import Flatten
from keras.layers import Dense

from keras.callbacks import CSVLogger

csv_logger = CSVLogger("DeepLearning_for_ImagingFlowCytometry/model_ad_history.csv", append=True)


train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

train_dataset = train.flow_from_directory("DeepLearning_for_ImagingFlowCytometry/data/adtrain",
                                          target_size=(28,28),
                                          batch_size = 10,
                                          class_mode = 'binary',
                                          color_mode='grayscale')
                                         
test_dataset = test.flow_from_directory("DeepLearning_for_ImagingFlowCytometry/data/adtest",
                                          target_size=(28,28),
                                          batch_size = 10,
                                          class_mode = 'binary',
                                          color_mode='grayscale')


model = keras.Sequential()

# Convolutional layer and maxpool layer 1
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2)) # just added

model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))



model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1, activation='sigmoid'))
    
    
    
    
    
# # Convolutional layer and maxpool layer 2
# model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
# model.add(keras.layers.MaxPool2D(2,2))



# #Convolutional layer and maxpool layer 4
# model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
# model.add(keras.layers.MaxPool2D(2,2))

# # This layer flattens the resulting image array to 1D array
# model.add(keras.layers.Flatten())

# # Hidden layer with 512 neurons and Rectified Linear Unit activation function 
# model.add(keras.layers.Dense(128,activation='relu'))

# # Output layer with single neuron which gives 0 for Cat or 1 for Dog 
# #Here we use sigmoid activation function which makes our model output to lie between 0 and 1
# model.add(keras.layers.Dense(1,activation='sigmoid'))


model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.fit_generator(train_dataset,
         steps_per_epoch = 8,
         epochs = 100,
         callbacks=[csv_logger],
         validation_data = test_dataset,
         #validation_steps=73
         
         )

Found 78 images belonging to 2 classes.
Found 34 images belonging to 2 classes.
Epoch 1/100


C:\Python\Python37\lib\site-packages\ipykernel_launcher.py:82: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


8/8 [==============================] - 1s 28ms/step - loss: 0.6934 - accuracy: 0.4872 - val_loss: 0.6951 - val_accuracy: 0.5000
Epoch 2/100
8/8 [==============================] - 0s 9ms/step - loss: 0.6849 - accuracy: 0.5000 - val_loss: 0.6839 - val_accuracy: 0.5000
Epoch 3/100
8/8 [==============================] - 0s 9ms/step - loss: 0.6716 - accuracy: 0.5000 - val_loss: 0.6763 - val_accuracy: 0.5000
Epoch 4/100
8/8 [==============================] - 0s 9ms/step - loss: 0.6491 - accuracy: 0.6026 - val_loss: 0.6550 - val_accuracy: 0.5882
Epoch 5/100
8/8 [==============================] - 0s 9ms/step - loss: 0.6167 - accuracy: 0.5769 - val_loss: 0.6132 - val_accuracy: 0.7647
Epoch 6/100
8/8 [==============================] - 0s 9ms/step - loss: 0.6521 - accuracy: 0.5513 - val_loss: 0.6707 - val_accuracy: 0.5000
Epoch 7/100
8/8 [==============================] - 0s 9ms/step - loss: 0.5334 - accuracy: 0.7821 - val_loss: 0.5594 - val_accuracy: 0.8529
Epoch 8/100
8/8 [=====================